In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import auc
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from scipy.stats import randint



from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import warnings

pd.set_option("display.max_columns", None)

warnings.filterwarnings("ignore")
#%load_ext nb_black

In [ ]:
data_train = pd.read_csv(
    "../input/tabular-playground-series-mar-2021/train.csv", sep=","
)
data_train.head()

In [ ]:
data_test = pd.read_csv(
    "../input/tabular-playground-series-mar-2021/test.csv", sep=","
)
data_test.head()

In [ ]:
data_train.shape
data_test.shape

In [ ]:
data_train.info()
data_test.info()

In [ ]:
data_train.drop(["id"], axis=1).hist(figsize=(15, 30), layout=(9, 3));

In [ ]:
for column in data_train.select_dtypes(include="object"):
    if data_train[column].nunique() < 30:
        sns.countplot(y=column, data=data_train)
        plt.show();

In [ ]:
plt.figure(figsize=(10, 8))
sns.boxplot(data=data_train.drop("id", axis=1), orient="h", palette="Set2")
plt.show();

In [ ]:
data_train_woid = data_train.drop(["id"], axis=1)
y_train = data_train.target
data_test_woid = data_test.drop(["id"], axis=1)

In [ ]:
data_train_woid.shape
data_test_woid.shape
y_train.shape

In [ ]:
cat_cols = [feature for feature in data_train_woid.columns if "cat" in feature]

le = preprocessing.LabelEncoder()

for enc in cat_cols:
    le = preprocessing.LabelEncoder()
    data_train_woid[enc] = le.fit_transform(data_train_woid[enc])
    data_test_woid[enc] = le.fit_transform(data_test_woid[enc])

In [ ]:
data_test_woid
data_train_woid

In [ ]:
data_for_corr = data_train_woid.corr()

In [ ]:
corr_matrix = data_for_corr
mask = np.zeros_like(corr_matrix, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
plt.figure(figsize=(18, 12))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap="coolwarm")
plt.show();

In [ ]:
###Plot all the variables correlated against target
draw = data_train_woid.drop("target", axis=1)
draw.corrwith(y_train).plot(kind="bar", grid=True, figsize=(12, 8))
plt.title("Correlation with target", size=20)
plt.xticks(size=10, rotation=60)
plt.yticks(size=10)
plt.show();

In [ ]:
X_train_final = data_train_woid.drop("target", axis=1)
X_train_final.head()

In [ ]:
y_train.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_train_final, y_train, test_size=0.25, shuffle=True, random_state=42
)

print("X_train - ", X_train.shape)
print("y_train - ", y_train.shape)
print("X_validation - ", X_test.shape)
print("y_validation - ", y_test.shape)

In [ ]:
###First XBGmodel
model = XGBClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(model, X_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1)
print("Mean ROC AUC: %.5f" % np.mean(scores))
model.fit(X_train,y_train)

In [ ]:
# Create the submission file
submission = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')
display(submission.head())
submission['target'] = model.predict(data_test_woid)
submission.to_csv('./xgboost_base_classifier.csv', index=False, header=True)

In [ ]:
###Drop categorical columns with too many values and create a new model
X_train_final_new = data_train_woid.drop(["target","cat5","cat7","cat8","cat10"], axis=1)
X_train_final_new.head()

In [ ]:
y_train_new = data_train.target
data_test_woid_new = data_test_woid.drop([ "cat5", "cat7", "cat8", "cat10"], axis=1)
data_test_woid_new.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train_final_new, y_train_new, test_size=0.25, shuffle=True, random_state=42
)

print("X_train - ", X_train.shape)
print("y_train - ", y_train.shape)
print("X_validation - ", X_test.shape)
print("y_validation - ", y_test.shape)

In [ ]:
model_new = XGBClassifier()
model_new.fit(X_train, y_train)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(model_new, X_train, y_train, scoring="roc_auc", cv=cv, n_jobs=-1)
print("Mean ROC AUC: %.5f" % np.mean(scores))

In [ ]:
y_predict = model_new.predict(X_test)
y_train_predict = model_new.predict(X_train)
roc_auc_score( y_train, y_train_predict)
roc_auc_score(y_test, y_predict)

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')
submission['target'] = model_new.predict(data_test_woid_new)
submission.to_csv('./xgboost_modelnew.csv', index=False, header=True)

In [ ]:
###One hot encoded models for 5 cat variables and remaining are label encoded

data_appended=data_train_woid.drop('target', axis=1).append(data_test_woid)

In [ ]:
cat_cols = [feature for feature in data_appended.columns if "cat" in feature]

le = preprocessing.LabelEncoder()

for enc in cat_cols:
    le = preprocessing.LabelEncoder()
    data_appended[enc] = le.fit_transform(data_appended[enc])

In [ ]:
encoded_data=pd.get_dummies(
    data_appended, columns=["cat5", "cat7", "cat8", "cat10"]
)

In [ ]:
data_train_ohc=encoded_data.iloc[0:300000]
data_test_ohc=encoded_data.iloc[300000:]

In [ ]:
y_train_new = data_train.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    data_train_ohc, y_train_new, test_size=0.25, shuffle=True, random_state=42
)

print("X_train - ", X_train.shape)
print("y_train - ", y_train.shape)
print("X_validation - ", X_test.shape)
print("y_validation - ", y_test.shape)

In [ ]:
model_enc = XGBClassifier()
model_enc.fit(X_train, y_train)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_val_score(model_enc, X_train, y_train, scoring="roc_auc", cv=cv, n_jobs=-1)
print("Mean ROC AUC: %.5f" % np.mean(scores))

In [ ]:
y_predict = model_enc.predict(X_test)
y_train_predict = model_enc.predict(X_train)
roc_auc_score( y_train, y_train_predict)
roc_auc_score(y_test, y_predict)

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')
submission['target'] = model_enc.predict(data_test_ohc)
submission.to_csv('./xgboost_modelenc.csv', index=False, header=True)

In [ ]:
###Use Random Search to improve the last model.
#I started this ompetition on the last day so there is no time to finish this step

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


model_rs = XGBClassifier()
param_grid_rand = {
    "learning_rate": [0.01, 0.05, 0.1, 0.5, 1],
    "max_depth": [3, 5, 10],
    "n_estimators": [100, 500, 1000],
    "min_child_weight": [5, 10],
    "gamma": [0.5, 1, 2],
    "subsample": [0.6, 0.8, 0.9],
    "random_state": [42],
}


model = RandomizedSearchCV(
    estimator=model_rs,
    param_distributions=param_grid_rand,
    n_iter=10,
    cv=3,
    verbose=2,
    random_state=42,
    n_jobs=-1,
)

model.fit(X_train,y_train)

RandomizedSearchCV(cv=3,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100,...
                                           random_state=None, reg_alpha=None,
                                           reg_lambda=None,
                                           scale_pos_weight=None,
                                           subsample=None, tree_method=None,
                                           validate_parameters=None,
                                           verbosity=None),
                   n_jobs=-1,
                   param_distributions={'gamma': [0.5, 1, 2],
                                        'learning_rate': [0.01, 0.05, 0.1, 0.5,
                                                          1],
                                        'max_depth': [3, 5, 10],
                                        'min_child_weight': [5, 10],
                                        'n_estimators': [100, 500, 1000],
                                        'random_state': [42],
                                        'subsample': [0.6, 0.8, 0.9]},
                   random_state=42, verbose=2)
                  

**Best model params**

{'subsample': 0.8,
 'random_state': 42,
 'n_estimators': 500,
 'min_child_weight': 5,
 'max_depth': 5,
 'learning_rate': 0.1,
 'gamma': 2}

In [ ]:
best_model=XGBClassifier(subsample=0.8,
 random_state=42,
 n_estimators= 500,
 min_child_weight= 5,
 max_depth= 5,
 learning_rate= 0.1,
 gamma= 2)

best_model.fit(X_train, y_train)

In [ ]:
y_predict = best_model.predict(X_test)
y_train_predict = best_model.predict(X_train)
roc_auc_score( y_train, y_train_predict)
roc_auc_score(y_test, y_predict)

In [ ]:
submission = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')
submission['target'] = best_model.predict(data_test_ohc)
submission.to_csv('./best_model.csv', index=False, header=True)